In [1]:
!pip install transformers

     |████████████████████████████████| 552kB 3.5MB/s 
     |████████████████████████████████| 3.7MB 52.7MB/s 
     |████████████████████████████████| 870kB 19.0MB/s 
     |████████████████████████████████| 1.0MB 33.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=8de31ea49ff340be9b3cd4838260064f0e2d370c3290b674d50365e768df7a0d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import BertModel,BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [0]:
encoded = tokenizer.encode('who was jim henson',add_special_tokens=True)


In [0]:
encoded



[101, 2040, 2001, 3958, 27227, 102]

In [0]:
indexed_tokens = torch.tensor([tokenizer.convert_tokens_to_ids(tokenized_text)])
indexed_tokens


NameError: ignored

In [0]:
indexed_tokens.shape

torch.Size([1, 14])

In [0]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

outputs = None
with torch.no_grad():
  outputs = model(indexed_tokens)
  
  

In [0]:
outputs[0].shape

torch.Size([1, 14, 768])

In [0]:
outputs[1].shape

torch.Size([1, 768])

In [0]:

question = ["How much is jacob paid",
            "What is the total salary of jacob",
          ]

cols = [  [ 'id','name','total salary' ], [ 'id','name','total salary'], ]





In [6]:
batch_question_word_id = []
batch_size = len(question)

for cur in question:
  batch_question_word_id.append(tokenizer.encode(cur,add_special_tokens=True))

print(batch_question_word_id)

maximum_length = max( len(x) for x in batch_question_word_id )
print(maximum_length)




[[101, 2129, 2172, 2003, 6213, 3825, 102], [101, 2054, 2003, 1996, 2561, 10300, 1997, 6213, 102]]
9


In [8]:
import numpy as np

batch_embedding = np.zeros((batch_size,maximum_length),dtype=np.int_)
for i in range(batch_size):

  batch_embedding[i,:len(batch_question_word_id[i])] = batch_question_word_id[i]

print(batch_embedding)

[[  101  2129  2172  2003  6213  3825   102     0     0]
 [  101  2054  2003  1996  2561 10300  1997  6213   102]]


In [9]:
question_embedding = torch.tensor(batch_embedding)
print(question_embedding.shape)

torch.Size([2, 9])


In [10]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

outputs = None
with torch.no_grad():
  outputs = model(question_embedding)
  

In [11]:
outputs[0].shape


torch.Size([2, 9, 768])

In [17]:
col_len = np.zeros((batch_size))
for i in range(batch_size):

  col_len[i] = len(cols[i])

print(col_len)

[3. 3.]


In [18]:
column_embeddings = [] 

with torch.no_grad():
  

  for b in range(batch_size):
    current_ques = []
    for c in cols[b]:
      encoded_name = torch.tensor([tokenizer.encode(c,add_special_tokens=True)])
      encoded = model(encoded_name)
      current_ques.append(encoded[1][0])

    column_embeddings.append(current_ques)


print(column_embeddings[0])






[tensor([-9.1390e-01, -2.6067e-01, -1.6177e-01,  8.6036e-01, -2.3931e-01,
        -9.2996e-02,  9.3982e-01,  1.4347e-01, -1.5215e-01, -9.9997e-01,
        -2.3034e-01,  7.5573e-01,  9.8750e-01,  1.5271e-01,  9.4250e-01,
        -6.4603e-01, -5.0243e-01, -4.9210e-01,  4.6132e-01, -8.5394e-01,
         6.6645e-01,  9.9549e-01,  5.4160e-01,  2.4272e-01,  4.8839e-01,
         9.0172e-01, -6.9724e-01,  9.5383e-01,  9.6663e-01,  7.4506e-01,
        -8.0295e-01,  1.2239e-01, -9.9282e-01, -2.8215e-01, -3.9581e-01,
        -9.9138e-01,  2.8494e-01, -8.3428e-01, -2.4879e-03,  8.8365e-02,
        -9.2930e-01,  2.3227e-01,  9.9976e-01,  3.2397e-01,  1.9170e-01,
        -3.5573e-01, -1.0000e+00,  2.1411e-01, -9.1079e-01,  4.2392e-01,
         2.2255e-01, -1.2050e-01,  1.2537e-01,  4.1669e-01,  5.1614e-01,
         3.8794e-01, -1.3235e-01,  1.2139e-01, -1.9305e-01, -6.1237e-01,
        -6.4926e-01,  5.1568e-01, -4.3699e-01, -8.9856e-01,  2.2986e-01,
         5.7683e-02, -1.2858e-01, -2.1665e-01, -3.

In [19]:
len(column_embeddings)

2

In [20]:
len(column_embeddings[0]
    )

3

In [21]:
column_embeddings[0][0].shape

torch.Size([768])

In [22]:
print(column_embeddings[0][1].shape)

torch.Size([768])


In [27]:
max_col_length  = max(col_len)
type(int(max_col_length))

int

In [0]:
max_col_length  = int(max(col_len))

column = torch.zeros((batch_size,max_col_length,768))

for b in range(batch_size):
  for l in range(int(col_len[b])):
    column[b,l,:] = column_embeddings[b][l]



In [31]:
column[0][0]

tensor([-9.1390e-01, -2.6067e-01, -1.6177e-01,  8.6036e-01, -2.3931e-01,
        -9.2996e-02,  9.3982e-01,  1.4347e-01, -1.5215e-01, -9.9997e-01,
        -2.3034e-01,  7.5573e-01,  9.8750e-01,  1.5271e-01,  9.4250e-01,
        -6.4603e-01, -5.0243e-01, -4.9210e-01,  4.6132e-01, -8.5394e-01,
         6.6645e-01,  9.9549e-01,  5.4160e-01,  2.4272e-01,  4.8839e-01,
         9.0172e-01, -6.9724e-01,  9.5383e-01,  9.6663e-01,  7.4506e-01,
        -8.0295e-01,  1.2239e-01, -9.9282e-01, -2.8215e-01, -3.9581e-01,
        -9.9138e-01,  2.8494e-01, -8.3428e-01, -2.4879e-03,  8.8365e-02,
        -9.2930e-01,  2.3227e-01,  9.9976e-01,  3.2397e-01,  1.9170e-01,
        -3.5573e-01, -1.0000e+00,  2.1411e-01, -9.1079e-01,  4.2392e-01,
         2.2255e-01, -1.2050e-01,  1.2537e-01,  4.1669e-01,  5.1614e-01,
         3.8794e-01, -1.3235e-01,  1.2139e-01, -1.9305e-01, -6.1237e-01,
        -6.4926e-01,  5.1568e-01, -4.3699e-01, -8.9856e-01,  2.2986e-01,
         5.7683e-02, -1.2858e-01, -2.1665e-01, -3.0